# Yeast Cell Mating Projection Growth with SpatialPy

## Definition of the model

### Imports and definitions

In [1]:
import os
import sys
import math
sys.path.insert(1, "../..")
import numpy as np
import matplotlib.pyplot as plt

import spatialpy

In [2]:
class Fluid(spatialpy.Geometry):
    def __init__(self, external_radius):
        self.ers = external_radius**2
        
    def create_particles(self, domain, xmin, xmax, ymin, ymax, delta):
        Nfp =0
        for x in np.arange(xmin, xmax+delta, delta):
            for y in np.arange(ymin, ymax+delta, delta):
                if self.inside((x, y), False):
                    domain.add_point([x, y, 0], type=0, vol=1, mass=1, nu=0, fixed=False)
                    Nfp += 1
        return Nfp
        
    def inside(self, x, on_boundary):
        return x[0]**2 + x[1]**2 > self.ers

In [3]:
class CellWall(spatialpy.Geometry):
    def __init__(self, external_radius, internal_radius):
        self.ers = external_radius**2
        self.irs = internal_radius**2
        
    def create_particles(self, domain, xmin, xmax, ymin, ymax, delta):
        Ncp = 0
        for x in np.arange(xmin, xmax+delta, delta):
            for y in np.arange(ymin, ymax+delta, delta):
                if self.inside((x, y), False):
                    domain.add_point([x, y, 0], type=0, vol=1, mass=1, nu=0, fixed=False)
                    Ncp += 1
        return Ncp
    
    def inside(self, x, on_boundary):
        radius_sqr = x[0]**2 + x[1]**2
        return radius_sqr < self.ers and radius_sqr > self.irs

In [4]:
class Interior(spatialpy.Geometry):
    def __init__(self, internal_radius):
        self.irs = internal_radius**2
        
    def create_particles(self, domain, xmin, xmax, ymin, ymax, delta):
        Nip = 0
        for x in np.arange(xmin, xmax+delta, delta):
            for y in np.arange(ymin, ymax+delta, delta):
                if self.inside((x, y), False):
                    domain.add_point([x, y, 0], type=0, vol=1, mass=1, nu=0, fixed=False)
                    Nip += 1
        return Nip
    
    def inside(self, x, on_boundary):
        return x[0]**2 + x[1]**2 < self.irs

### Model

In [7]:
class YeastCellMatingProjectionGrowth(spatialpy.Model):
    FLUID = 1
    CELL_WALL = 2
    INTERIOR = 3
    
    def __init__(self, model_name="Yeast Cell Mating Projection Growth"):
        spatialpy.Model.__init__(self, model_name)
        
        # System Constants
        L = 50e-6
        radius_external = 0.2*L
        radius_internal = 0.15*L
        rhof = 1000
        rhoc = 1100
        rhoi = 1500
        
        # Species
        
        # Discretizations
        Npx = 100
        
        # Compute domain bounds (including the boundary)
        deltaf = L/Npx
        deltac = 0.8*deltaf
        xmin = ymin = -L/2
        xmax = ymax = L/2
        
        # Domain
        domain = spatialpy.Domain(
            0, xlim=(xmin, xmax), ylim=(ymin, ymax), zlim=(0, 0), gravity=[0, -1, 0]
        )
        
        # Fluid Region
        fluid_region = Fluid(radius_external)
        Nfp = fluid_region.create_particles(domain, xmin, xmax, ymin, ymax, deltaf)
        # Cell Wall Region
        cell_region = CellWall(radius_external, radius_internal)
        Ncp = cell_region.create_particles(
            domain, -radius_external, radius_external, -radius_external, radius_external, deltac
        )
        # Cell Interior Region
        interior_region = Interior(radius_internal)
        Nip = interior_region.create_particles(domain, xmin, xmax, ymin, ymax, deltaf)
        
        self.add_domain(domain)
        
        # Compute volume and mass per particle
        vol = L**2
        Cvol = math.pi*radius_external**2
        CIvol = math.pi*radius_internal**2
        CWvol = Cvol-CIvol
        Fvol = vol-Cvol
        
        # Types
        self.set_type(fluid_region, self.FLUID)
        self.set_type(cell_region, self.CELL_WALL)
        self.set_type(interior_region, self.INTERIOR)
        
        # Timespan
        dt = 1e-10
        nt = 40000
        freq_results = 1000
        self.timespan(np.arange(0, nt*dt+dt, freq_results*dt), timestep_size=dt)

In [8]:
model = YeastCellMatingProjectionGrowth()

In [10]:
model.domain.plot_types()

In [11]:
set(model.domain.vol)

{1.0}

In [12]:
model.domain.calculate_vol()
set(model.domain.vol)

AttributeError: 'NoneType' object has no attribute 'shape'